<a href="https://colab.research.google.com/github/isaranwrap/AKITomorrow/blob/master/AKITomorrow2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

Trifecta + datetime for time manipulation

In [0]:
#trifecta
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#to check location & lab vals @ time of alert
from datetime import datetime, timedelta

# User-defined functions

---
> ### returnAlertCounter - returns the counter of the row at which the latest time before the alert occurs
>> Input: alerttime (datetimeformat), dt_list (list of datetimes), & optional timedelta (timedelta, default = 0 hours)

>> Output: index (integer) in dt_list (list of datetimes) at which latest vals before alerttime are 

> ### returnNoLongerERCounter - returns counter at which patient is no longer in the ER

>> Input: encounter (integer, pat_enc_csn_id) of interest

>> Output: index (integer) in final_locations dataframe when patient is no longer in ER

> ### getAge - returns the age of the patient given birth_date & admission_date. Some interesting consequences associated with timedelta returning in days and the Logistic regression model ceiling in years make a user-defined function necessary. 

>> Input: birth_date & admission_date in datetime format

>> Output: age of patient (integer) already ceiling'd (for LogReg model)

In [0]:
def returnAlertCounter(alerttime, dt_list, window = timedelta(hours=0)):
  counter = 0
  try:
    while alerttime + window >= dt_list[counter]:
      counter += 1
    return counter - 1
  except IndexError:
    return counter - 1

def returnNoLongerERCounter(enc):
  counter = 0
  try:
    while final_locs[final_locs.PAT_ENC_CSN_ID == enc].iloc[counter].Emergency_Room:
      counter += 1
    return counter
  except IndexError:
    return counter-1

def getAge(birth_date, admission_date):
    years = admission_date.year - birth_date.year
    months = admission_date.month - birth_date.month
    days = admission_date.day - birth_date.day
    if days == 0 and months == 0:
        return years
    else:
        if days < 0:
            months -= 1
        if months < 0:
            years -= 1
        return years + 1

# Loading in datasets

---
> ### cleanlabs | 316764 x 12 | lab values of patients (s.a. hemoglobin, creatinine & BUN)

> ### patients | 2850 x 5 | patient MRN #, encounter ID, alert_time & admission/discharge times

> ### demo | 7362 x 13 | demographic information of patients

> ### final_locs | 39744 x 12 | locations of the patients @ time of transfer

In [0]:
baseFolder = ''
clean_labs = pd.read_csv(baseFolder + 'cleanlab.csv') #316764 x 12,   #6090 unique patients  #6982 unique patient encounters
patients = pd.read_csv(baseFolder + 'alerted patients.csv') #2850 x 5 the 2850 patients who were alerted #2850 x 5 --> pat_mrn, pat_enc, alert_time, adm, dis
demo = pd.read_csv(baseFolder + 'demo.csv') # 7362 x 13 demographics info (7362 unique encounters)
final_locs = pd.read_csv(baseFolder + 'finallocation.csv') # the locations of the patients throughout their hospital trip 39744 x 12

# Preprocessing Slicing/Reshaping datasets 
--- 
> ### patients | turn PAT_MRN_ID & PAT_ENC_CSN_ID to lowercase, convert admission to datetime format & extract just date part of that (admit_date col) 
>> create list of alerted_encounters based on patients df 

> cleanlabs | perform forward imputation on labs, subset rows by those in alerted_encounters

> final_locs | subset final_locs by those in alerted_encounters

> demo | subset demo by those in alerted_encounters & cols we care about (race, birth_date & sex) & sort pat_mrn_id to match patients df 

In [0]:
#capitalize the patients dataframe for later dropping duplicates during concatenation - need same col names
patients['PAT_MRN_ID'] = patients['pat_mrn_id']
patients['PAT_ENC_CSN_ID'] = patients['pat_enc_csn_id']
patients.drop(['pat_mrn_id','pat_enc_csn_id'],axis=1, inplace=True)
patients['admission'] = pd.to_datetime(patients['admission'])
patients['admit_date'] = [i.date() for i in patients['admission']]
patients['admit_date'] = pd.to_datetime(patients['admit_date'])
patients['alert_time'] = pd.to_datetime(patients['alert_time'])
alerted_encounters = list(patients.PAT_ENC_CSN_ID) #we care at the level of encounter

#perform forward imputation on clean labs, for some reason groupby drops col so loaded in dataset it just tmp
cleanlabs = clean_labs.groupby('PAT_MRN_ID').ffill()
cleanlabs['PAT_MRN_ID'] = clean_labs['PAT_MRN_ID']

cleanlabs_dtformat = loc_dtformat =  '%d%b%y:%H:%M:%S'
alerttime_dtformat = '%Y-%m-%dT%H:%M:%S'

cleanlabs['RESULT_TIME'] = pd.to_datetime(cleanlabs.RESULT_TIME, format=cleanlabs_dtformat)
#subsetting for cleanlabs by patient takes a bit of work, so that's down below
#...but, we can subset by rows
cleanlabs_rows = cleanlabs.PAT_ENC_CSN_ID.isin(alerted_encounters)
#cleanlabs = cleanlabs.loc[cleanlabs_rows,:]

#whereas subsetting final_locations is easier
finallocs_rows = final_locs.PAT_ENC_CSN_ID.isin(alerted_encounters)
final_locs = final_locs.loc[finallocs_rows]
final_locs['INSTANT_PT_MOVED_TO_DEPTRMBED'] = pd.to_datetime(final_locs['INSTANT_PT_MOVED_TO_DEPTRMBED'], format = loc_dtformat)

#subsetting demographics we care about & sorting according to how patients df is sorted
#demo[demo.PAT_ENC_CSN_ID.isin(alerted_encounters)]
demo_cols = ['PAT_MRN_ID',
             'PAT_ENC_CSN_ID',
             'BIRTH_DATE',
             'RACE',
             'SEX']
demo_rows = demo.PAT_ENC_CSN_ID.isin(alerted_encounters)
demo = demo[demo_cols][demo_rows] #shape is 2850 x 5 --> birth_date, race & sex
demo['BIRTH_DATE'] = pd.to_datetime(demo['BIRTH_DATE'])
demo.sort_values('PAT_MRN_ID', inplace=True)
demo.reset_index(drop=True, inplace=True)

# Subsetting clean labs 
---
> clean labs we care about: HCO3-, Hg, BUN, Cl-, Creat, K+, Na+
> & find row which has the latest value BEFORE the alert time
>> Forward imputation has already been implemented in df, it's just a matter of extracting the most current lab vals before alert_time, which is taken from a different dataframe, so to conv to datetime & compare is a little convoluted but done below. I keep track of patient_indx as well as alert_indx, reset the indices, & subset cols by the clean_labs we care about + patient info

In [0]:
cleanlab_cols = ['PAT_MRN_ID',
                 'PAT_ENC_CSN_ID',
                 'bicarbonate',
                 'hemoglobin',
                 'bun',
                 'chloride',
                 'creatinine',
                 'potassium',
                 'sodium',
                 'RESULT_TIME'] 


#time @ which lab development occurred, in datetime format. I now need to find the dt which corresponds to alert_time
#turn alert_time to dt format as well for comparison purposes

enc_labs = list()
for indx, enc in enumerate(alerted_encounters):
  #cleanlabs.iloc[80:100], datetime.strptime(patients.loc[patients.pat_enc_csn_id == enc].alert_time[indx], '%Y-%m-%dT%H:%M:%S')
  #alerttime_dt = datetime.strptime(patients.loc[patients.PAT_ENC_CSN_ID == enc].alert_time[indx], alerttime_dtformat) old, before datetime conv
  alerttime = patients.loc[patients.PAT_ENC_CSN_ID == enc].alert_time[indx]
  #cleanlabs_RESULTTIME = [datetime.strptime(date, cleanlabs_dtformat) for date in cleanlabs[cleanlabs.PAT_ENC_CSN_ID == enc].RESULT_TIME] old, before datetime conv
  cleanlabs_RESULTTIME = list(cleanlabs[cleanlabs.PAT_ENC_CSN_ID == enc].RESULT_TIME)
  alert_indx = returnAlertCounter(alerttime = alerttime, dt_list = cleanlabs_RESULTTIME)#, window = timedelta(hours=2)) #comment this line out to take away 2 hr window
  pat_indx = cleanlabs.index[cleanlabs.PAT_ENC_CSN_ID == enc][0]
  enc_labs.append(cleanlabs.iloc[[pat_indx + alert_indx]])

cleanlabs_df = pd.concat(enc_labs) #2850 x 12
cleanlabs_indices = cleanlabs_df.index
cleanlabs_df.reset_index(drop=True, inplace=True)
cleanlabs_df = cleanlabs_df.loc[:,cleanlab_cols] #2850 x 9

# Merge demographics + clean labs into a big temporary data frame - also a good checkpoint

--- 
> ### Drop the mismatches from patients & clean labs, then combine into a temporary big dataframe with more columns than need be. Aso convert alert_time & birth_date into datetime (which is stupid & redundant, should have done this differently in retrospect, all @ once in the beginning), &  create age, sex, and race vars by how the Logistic Regression model takes it in.
>>  For some reason there are some (~10 oom) mismatches which I just drop - don't care to look into them... I'm a busy college student okay?



In [0]:
mismatch = np.where(np.array(patients.PAT_MRN_ID) != np.array(cleanlabs_df.PAT_MRN_ID))[0]
patients.drop(mismatch, inplace= True)
cleanlabs_df.drop(mismatch, inplace=True)
demo.drop(mismatch, inplace=True)
#tmp_big_df = pd.concat([patients, cleanlabs_df], axis=1)
#tmp_big_df = tmp_big_df.loc[:,~tmp_big_df.columns.duplicated()]
tmp_big_df = patients.merge(cleanlabs_df)
tmp_big_df = tmp_big_df.merge(demo)


#Checks
#np.all(np.array(demo.PAT_MRN_ID) == np.array(patients.PAT_MRN_ID))
#np.all(np.array(patients.PAT_MRN_ID) == np.array(cleanlabs_df.PAT_MRN_ID))

#No longer necessary if you do it in the beginning
#tmp_big_df['alert_time'] = pd.to_datetime(tmp_big_df['alert_time'])
#tmp_big_df['BIRTH_DATE'] = pd.to_datetime(tmp_big_df['BIRTH_DATE'])

#tmp_big_df['age'] = np.ceil((tmp_big_df['alert_time'] - tmp_big_df['BIRTH_DATE']).apply(lambda x: x/timedelta(days=365)))# -1st
#tmp_big_df['age'] = np.ceil((tmp_big_df['admit_date'] - tmp_big_df['BIRTH_DATE']).apply(lambda x: x/timedelta(days=365))) # -2nd w admit date
#mp_big_df['age'] = [relativedelta(i, j) for i, j in zip(tmp_big_df['admit_date'],tmp_big_df['BIRTH_DATE'])]#']), tmp_big_df['BIRTH_DATE']) 
#Adi's code 
tmp_big_df['age'] = [getAge(tmp_big_df['BIRTH_DATE'][i],tmp_big_df['admission'][i]) for i in range(tmp_big_df.shape[0])]
tmp_big_df['sex'] = tmp_big_df['SEX'] == 'Male'
tmp_big_df['race'] = tmp_big_df['RACE'] == 'Black or African American'
tmp_big_df.head() #shape is 2840 x 19 

# Create final_locations .csv -- where patients end up (after ER), divided into surgical, medical, icu, etc. 

In [0]:
#DON'T RUN THIS IF WHAT YOU'RE TRYNA DO IS get risk score - icu & surgical vars ARE DIFF!
#Uncomment code below if & only if you are interested in finding the final locations, i.e. where these patients END up 
#Convert to True/False
#for col in final_locs.columns[4:]:
  #final_locs[col] = final_locs[col] == 1.0

#for enc in alerted_encounters:
  #row = final_locs[final_locs.PAT_ENC_CSN_ID == enc].iloc[returnNoLongerERCounter(enc)]
  #tmp_big_df.loc[tmp_big_df.PAT_ENC_CSN_ID == enc, 'medical'] = row.Medical_ICU or row.Medical_Ward or row.Medical_Surgical_Ward or row.Emergency_Room
  #tmp_big_df.loc[tmp_big_df.PAT_ENC_CSN_ID == enc, 'surgical'] = row.Surgery_Ward or row.Surgical_ICU or row.Medical_Surgical_ICU or row.Medical_Surgical_Ward
  #tmp_big_df.loc[tmp_big_df.PAT_ENC_CSN_ID == enc, 'icu'] = row.Medical_ICU or row.Surgical_ICU or row.Medical_Surgical_ICU
  #tmp_big_df.loc[tmp_big_df.PAT_ENC_CSN_ID == enc, 'admit_medical'] = row.Emergency_Room

# Create surgical & icu variables to be put into Logistic Regression model
---
> ### icu is True if the patient is in Medical_ICU, Surgical_ICU, or Medical_Surgical_ICU @ time of alert
> ### surgical is True if the patient is in Surgery_ward, Surgical_ward (IDK THE DIFFERENCE! x) Surgical_ICU, Medical_Surgical_ICU (so there should be some overlap) or Medical_Surgical_Ward @ time of alert

In [0]:
#Convert to True/False
for col in final_locs.columns[4:]:
  final_locs[col] = final_locs[col] == 1.0

for indx, enc in enumerate(alerted_encounters):
  if indx not in mismatch: #interesting consequence of dropping 10 patients is error gets thrown on key 706(first mismatch)
    alerttime = patients.loc[patients.PAT_ENC_CSN_ID == enc].alert_time[indx]
    #loc_dtlist = [datetime.strptime(date, loc_dtformat) for date in final_locs[final_locs.PAT_ENC_CSN_ID == enc].INSTANT_PT_MOVED_TO_DEPTRMBED]
    loc_dtlist = list(final_locs[final_locs.PAT_ENC_CSN_ID == enc].INSTANT_PT_MOVED_TO_DEPTRMBED)
    row = final_locs[final_locs.PAT_ENC_CSN_ID == enc].iloc[returnAlertCounter(alerttime, dt_list = loc_dtlist)]
    tmp_big_df.loc[tmp_big_df.PAT_ENC_CSN_ID == enc, 'icu'] = row.Medical_ICU or row.Surgical_ICU or row.Medical_Surgical_ICU 
    tmp_big_df.loc[tmp_big_df.PAT_ENC_CSN_ID == enc, 'surgical'] = row.Surgery_Ward or row.Surgical_Ward or row.Surgical_ICU or row.Medical_Surgical_ICU or row.Medical_Surgical_Ward

# Calculating risk score
---
> ### Beta coefficients given by previously trained LogReg model, supplied by Yu

In [0]:
beta_coeffs = [
               0.0090205, #age
               0.0028409, #bun
               0.3149055, #creat
               0.2605861, #potassium
              -0.0768276, #bicarbonate
              -0.0606976, #chloride 
              -0.0115591, #hemoglobin
               0.0442439, #sodium
               0.0274777, #sex
               0.1555640, #race
               0.9760579, #ICU
               0.2331921, #surgical
]
vars = ['age',
        'bun',
        'creatinine',
        'potassium',
        'bicarbonate',
        'chloride',
        'hemoglobin',
        'sodium',
        'sex',
        'race',
        'icu',
        'surgical'
]
pat_enc = tmp_big_df[['PAT_MRN_ID', 'PAT_ENC_CSN_ID']]
vars_df = tmp_big_df[vars]

#Final check before:Final check before:
risk_score = np.zeros(vars_df.shape[0])
for indx,col in enumerate(vars_df.columns):
  risk_score += beta_coeffs[indx]*vars_df[col]
vars_df['risk_score'] = risk_score - 3.333648 #one last beta coeff
df = pd.concat([pat_enc, vars_df], axis=1)
df.to_csv('RiskScores.csv', index=False)

#To check if two dataframes are the same, 
#np.where(pdf != df)[0].shape[0] == pdf.isnull().sum().sum()
# is the check I use. Because NaN shows up as not being the same thing in equality comparisons
#OR
#(pdf.iloc[np.where(pdf != df)[0]].round(decimals=5) != df.iloc[np.where(pdf != df)[0]].round(decimals=5)).sum().sum() == (pdf.iloc[np.where(pdf != df)[0]]).isnull().sum().sum()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
subset = df[df.risk_score < -1.73] #these are the patients below the threshold who nonetheless alerted.... why? 
below_threshold = list(subset.PAT_ENC_CSN_ID) 

below_thresh = list()
for indx, enc in enumerate(below_threshold):
  alerttime = patients[patients.PAT_ENC_CSN_ID == enc].alert_time[patients.index[patients.PAT_ENC_CSN_ID == enc][0]]
  cleanlabs_RESULTTIME = list(cleanlabs[cleanlabs.PAT_ENC_CSN_ID == enc].RESULT_TIME)
  alert_indx = returnAlertCounter(alerttime = alerttime, dt_list = cleanlabs_RESULTTIME)#, window = timedelta(hours=2)) #comment this line out to take away 2 hr window
  front_window = returnAlertCounter(alerttime = alerttime, dt_list = cleanlabs_RESULTTIME, window=timedelta(hours=-2))+1 #add plus one, first time AFTER -2 hr window
  back_window = returnAlertCounter(alerttime = alerttime, dt_list = cleanlabs_RESULTTIME, window=timedelta(hours=2))+1 # last time BEFORE +2 hr window 
  pat_indx = cleanlabs.index[cleanlabs.PAT_ENC_CSN_ID == enc][0]
  if front_window != back_window:
    below_thresh.append(cleanlabs[cleanlabs.PAT_ENC_CSN_ID == enc].iloc[front_window:back_window])

bt = pd.concat(below_thresh)
bt.reset_index(drop=True, inplace=True)
bt = bt.loc[:,cleanlab_cols] 

In [0]:
tmp2 = patients.merge(bt)
tmp2 = tmp2.merge(demo)

tmp2['age'] = [getAge(tmp2['BIRTH_DATE'][i],tmp2['admission'][i]) for i in range(tmp2.shape[0])]
tmp2['sex'] = tmp2['SEX'] == 'Male'
tmp2['race'] = tmp2['RACE'] == 'Black or African American'
tmp2['icu'] = np.zeros(tmp2.shape[0],dtype=bool)
tmp2['surgical'] = np.zeros(tmp2.shape[0],dtype=bool)
icu_indx = tmp2.columns.get_loc('icu')
surg_indx = tmp2.columns.get_loc('surgical')
for enc in below_threshold: 
  loc_dtlist = list(final_locs[final_locs.PAT_ENC_CSN_ID == enc].INSTANT_PT_MOVED_TO_DEPTRMBED)
  for indx in tmp2[tmp2.PAT_ENC_CSN_ID == enc].index:
    alerttime = tmp2[tmp2.PAT_ENC_CSN_ID == enc].RESULT_TIME[indx]
    row = final_locs[final_locs.PAT_ENC_CSN_ID == enc].iloc[returnAlertCounter(alerttime, dt_list = loc_dtlist)]
    tmp2.iloc[indx,icu_indx] = row.Medical_ICU or row.Surgical_ICU or row.Medical_Surgical_ICU
    tmp2.iloc[indx,surg_indx] = row.Surgery_Ward or row.Surgical_Ward or row.Surgical_ICU or row.Medical_Surgical_ICU or row.Medical_Surgical_Ward

In [0]:
#for indx, enc in enumerate(below_threshold): 
 # alerttime = patients.loc[patients.PAT_ENC_CSN_ID == enc].alert_time[patients.index[patients.PAT_ENC_CSN_ID == enc][0]]
  ##loc_dtlist = [datetime.strptime(date, loc_dtformat) for date in final_locs[final_locs.PAT_ENC_CSN_ID == enc].INSTANT_PT_MOVED_TO_DEPTRMBED]
 # loc_dtlist = list(final_locs[final_locs.PAT_ENC_CSN_ID == enc].INSTANT_PT_MOVED_TO_DEPTRMBED)
 # front_window = returnAlertCounter(alerttime = alerttime, dt_list = loc_dtlist, window=timedelta(hours=-2))+1 #add plus one, first time AFTER -2 hr window
 # back_window = returnAlertCounter(alerttime = alerttime, dt_list = loc_dtlist, window=timedelta(hours=2))+1 # last time BEFORE +2 hr window 
 # row = final_locs[final_locs.PAT_ENC_CSN_ID == enc].iloc[returnAlertCounter(alerttime, dt_list = loc_dtlist)]
 # tmp2.loc[tmp2.PAT_ENC_CSN_ID == enc, 'icu'] = row.Medical_ICU or row.Surgical_ICU or row.Medical_Surgical_ICU 
 # tmp2.loc[tmp2.PAT_ENC_CSN_ID == enc, 'surgical'] = row.Surgery_Ward or row.Surgical_Ward or row.Surgical_ICU or row.Medical_Surgical_ICU or row.Medical_Surgical_Ward
#tmp2.head()

In [0]:
pat_enc = tmp2[['PAT_MRN_ID', 'PAT_ENC_CSN_ID']]
vars_df = tmp2[vars]

#Final check before:Final check before:
risk_score = np.zeros(vars_df.shape[0])
for indx,col in enumerate(vars_df.columns):
  risk_score += beta_coeffs[indx]*vars_df[col]
vars_df['risk_score'] = risk_score - 3.333648 #one last beta coeff
df2 = pd.concat([pat_enc, vars_df], axis=1)
alerted = df2[df2.risk_score > -1.73]
alerted.to_csv('RiskScores[alerted+-2hrs].csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
